In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd

## Supplier / Receiver pharmacy list (5/11 기준)

In [0]:
# Prediction 없는 특별 처리 대상 약국
exceptions = [12862631, 12817589, 11870907, 12866903, 12849634]

In [0]:
PATH1 = './DATA for optimization prob/'

In [0]:
with open(PATH1+"pharmacy_list.txt", 'r') as f:
  pharmacy_list = [int(code[:-1]) for code in f.readlines()]    # 서초구 내 전체 약국

with open(PATH1+"supplier_list.txt", 'r') as f:
  supplier_list = [int(code[:-1]) for code in f.readlines()]    # 5월 11일 14:00 ~ 15:00 PLENTY 약국

with open(PATH1+"receiver_list.txt", 'r') as f:
  receiver_list = [int(code[:-1]) for code in f.readlines()]    # 5월 11일 14:00 ~ 15:00 EMPTY/FEW 약국

## 서초구 전체 데이터 for Distance & CurrentInventory

In [0]:
PATH2 = './Data/district_0314-0512/'

In [0]:
# 서초구 전체
SEOCHO_df = pd.read_csv(PATH2+'서초구_0314-0512.csv', index_col=0, header=0, engine='python', encoding='utf-8-sig')

In [8]:
SEOCHO_df.isna().sum()

name             0
addr             0
latitude         0
longitude        0
collect_at     128
remain_stat    128
stock_at       128
dtype: int64

In [0]:
SEOCHO_df.dropna(inplace=True)

In [0]:
SEOCHO_df['collect_at'] = pd.to_datetime(SEOCHO_df['collect_at'])
SEOCHO_df['stock_at'] = pd.to_datetime(SEOCHO_df['stock_at'])

In [0]:
SEOCHO_df['date'] = SEOCHO_df['collect_at'].dt.date

In [0]:
SEOCHO_df.drop(columns=['addr', 'stock_at'], inplace=True)

In [0]:
SEOCHO_df = SEOCHO_df[['name', 'date', 'collect_at', 'remain_stat', 'latitude', 'longitude']]

In [0]:
# 5월 11일 서초구 데이터
SEOCHO_0511 = SEOCHO_df[SEOCHO_df.date==pd.to_datetime('2020-05-11')]

In [15]:
SEOCHO_0511

,name,date,collect_at,remain_stat,latitude,longitude
code,,,,,,
11800208,서초삼성약국,2020-05-11,2020-05-11 00:00:00,empty,37.495623,127.027107
11800208,서초삼성약국,2020-05-11,2020-05-11 00:05:00,empty,37.495623,127.027107
11800208,서초삼성약국,2020-05-11,2020-05-11 00:10:00,empty,37.495623,127.027107
11800208,서초삼성약국,2020-05-11,2020-05-11 00:15:00,empty,37.495623,127.027107
11800208,서초삼성약국,2020-05-11,2020-05-11 00:20:00,empty,37.495623,127.027107
...,...,...,...,...,...,...
12869414,8층약국,2020-05-11,2020-05-11 23:35:00,empty,37.492993,127.013451
12869414,8층약국,2020-05-11,2020-05-11 23:40:00,empty,37.492993,127.013451
12869414,8층약국,2020-05-11,2020-05-11 23:45:00,empty,37.492993,127.013451


In [16]:
SEOCHO_df[SEOCHO_df.index.isin([11800208, 12870510])]

,name,date,collect_at,remain_stat,latitude,longitude
code,,,,,,
11800208,서초삼성약국,2020-03-14,2020-03-14 00:00:00,empty,37.495623,127.027107
11800208,서초삼성약국,2020-03-14,2020-03-14 00:05:00,empty,37.495623,127.027107
11800208,서초삼성약국,2020-03-14,2020-03-14 00:10:00,empty,37.495623,127.027107
11800208,서초삼성약국,2020-03-14,2020-03-14 00:15:00,empty,37.495623,127.027107
11800208,서초삼성약국,2020-03-14,2020-03-14 00:20:00,empty,37.495623,127.027107
...,...,...,...,...,...,...
11800208,서초삼성약국,2020-05-11,2020-05-11 23:35:00,break,37.495623,127.027107
11800208,서초삼성약국,2020-05-11,2020-05-11 23:40:00,break,37.495623,127.027107
11800208,서초삼성약국,2020-05-11,2020-05-11 23:45:00,break,37.495623,127.027107


## Demand prediction result

In [0]:
PATH3 = './Data/Demand/Prediction/'

In [0]:
demand_preds = pd.read_csv(PATH3+'SEOCHO_Demand_Prediction.csv', index_col='code', header=0, engine='python', encoding='utf-8-sig')

In [0]:
demand_preds['date'] = pd.to_datetime(demand_preds['date'])

In [0]:
# 5월 11일 demand predictions
demand_preds_0511 = demand_preds[demand_preds.date==pd.to_datetime('2020-05-11')]

In [24]:
demand_preds_0511

,date,rate_pred,demand_pred
code,,,
12830721,2020-05-11,11.110000,71.309535
12860417,2020-05-11,25.913333,143.663520
12848131,2020-05-11,11.110000,54.994500
12816388,2020-05-11,12.766667,84.821733
12865028,2020-05-11,11.110000,79.436500
...,...,...,...
12816370,2020-05-11,20.555000,146.063830
11803126,2020-05-11,37.535000,228.738290
12867063,2020-05-11,69.415000,496.317250


In [0]:
# supplier 50개 선정
supplier_candidates = demand_preds_0511[demand_preds_0511.index.isin(supplier_list)]

In [0]:
final_suppliers = supplier_candidates.sort_values(by=['rate_pred']).iloc[:50]

In [42]:
len(final_suppliers)

50

In [0]:
final_supplier_list = list(final_suppliers.index)

In [0]:
receivers = demand_preds_0511[demand_preds_0511.index.isin(receiver_list)]

In [0]:
final_receiver_list = list(receivers.index)

In [0]:
#최종 Demand Rate 예측 값 테이블 생성
DEMAND_RATE_SUPPLIERS = final_suppliers['rate_pred']
DEMAND_RATE_RECEIVERS = receivers['rate_pred']

In [0]:
DEMAND_RATE_SUPPLIERS.to_csv(PATH1+'DEMAND_RATE_PREDICTION_TABLE_suppliers.csv', encoding='utf-8-sig', index=True)
DEMAND_RATE_RECEIVERS.to_csv(PATH1+'DEMAND_RATE_PREDICTION_TABLE_receivers.csv', encoding='utf-8-sig', index=True)

with open(PATH1+"final_supplier_list.txt", 'w') as output:
    for code in final_supplier_list:
        output.write(str(code) + '\n')
with open(PATH1+"final_receiver_list.txt", 'w') as output:
    for code in final_receiver_list:
        output.write(str(code) + '\n')

## Inventory status TABLE at 14 pm

In [0]:
SEOCHO_0511_receivers = SEOCHO_0511[SEOCHO_0511.index.isin(final_receiver_list)]

In [0]:
INVENTORY_receivers = SEOCHO_0511_receivers[(SEOCHO_0511_receivers['collect_at']>='2020-05-11 14:00:00') & (SEOCHO_0511_receivers['collect_at']<='2020-05-11 15:00:00')]

In [0]:
INVENTORY_receivers = INVENTORY_receivers[INVENTORY_receivers.remain_stat.isin(['empty', 'few'])]

In [0]:
INVENTORY_receivers.drop_duplicates('name', inplace=True)

In [0]:
INVENTORY_receivers = INVENTORY_receivers[['name', 'collect_at', 'remain_stat']]

In [0]:
INVENTORY_receivers['current_inventory'] = CURRENT_INVENTORY_receivers['remain_stat'].apply(lambda x: 0 if x == 'empty' else 15)

In [0]:
#최종 Inventory 상태 값 테이블 생성
CURRENT_INVENTORY_receivers = INVENTORY_receivers[['name', 'collect_at', 'current_inventory']]
CURRENT_INVENTORY_receivers.to_csv(PATH1+'CURRENT_INVENTORY_TABLE_receivers.csv', encoding='utf-8-sig', index=True)

## Distance Matrix

In [0]:
players_list = final_supplier_list+final_receiver_list

In [170]:
len(players_list)   # 70 by 70

70

In [0]:
players_df = SEOCHO_0511[SEOCHO_0511.index.isin(players_list)]

In [172]:
players_df.drop_duplicates('name', inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [174]:
players_df['coords'] = players_df.apply(lambda row: (row['latitude'], row['longitude']), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
players_df = players_df[['latitude', 'longitude', 'coords']]

In [178]:
np.array(players_df['coords'])

array([(37.4997906, 127.0231824), (37.4821718, 126.9969232),
       (37.4849497, 127.0147775), (37.4828549, 127.008763),
       (37.5060987, 127.00341740000002), (37.5071711, 127.02270459999998),
       (37.4865092, 127.0209709), (37.4905531, 127.0212546),
       (37.4944451, 127.0132771), (37.4968205, 126.9845952),
       (37.49383529999999, 126.9832127), (37.50450395, 127.0005993),
       (37.4964722, 126.9853016), (37.5075097, 127.0080778),
       (37.5066094, 127.0226368), (37.4972231, 127.0262314),
       (37.49515770000001, 127.0176552), (37.4944061, 127.0151321),
       (37.50440748, 127.0010481), (37.45714870000001, 127.015332),
       (37.508788, 127.0035627), (37.4850956, 126.9949469),
       (37.5070108, 127.006438), (37.5038061, 127.0241492),
       (37.4928604, 127.0141471), (37.4884264, 127.0141123),
       (37.4843263, 127.0379729), (37.49476, 127.016321),
       (37.5044991, 127.0056986), (37.4870229, 126.9828023),
       (37.4904873, 127.0087491), (37.5006686, 127.0032

In [0]:
distance_df = pd.DataFrame(index=players_df.index, columns=players_df.index)

In [182]:
list(enumerate(players_df.index))

[(0, 11803011),
 (1, 11803126),
 (2, 11808390),
 (3, 11822821),
 (4, 11824719),
 (5, 11828111),
 (6, 11848391),
 (7, 11854031),
 (8, 11854359),
 (9, 11854740),
 (10, 11854855),
 (11, 11855762),
 (12, 11855789),
 (13, 11855924),
 (14, 11874813),
 (15, 11878304),
 (16, 11882492),
 (17, 11883847),
 (18, 11884835),
 (19, 11887982),
 (20, 12806081),
 (21, 12808733),
 (22, 12808946),
 (23, 12810801),
 (24, 12811068),
 (25, 12812170),
 (26, 12814164),
 (27, 12816388),
 (28, 12817732),
 (29, 12818917),
 (30, 12822922),
 (31, 12823210),
 (32, 12824780),
 (33, 12824950),
 (34, 12829871),
 (35, 12830721),
 (36, 12830879),
 (37, 12831905),
 (38, 12833398),
 (39, 12833924),
 (40, 12837962),
 (41, 12839728),
 (42, 12841528),
 (43, 12842664),
 (44, 12842800),
 (45, 12844152),
 (46, 12845345),
 (47, 12848131),
 (48, 12850047),
 (49, 12850713),
 (50, 12850837),
 (51, 12851884),
 (52, 12852848),
 (53, 12853372),
 (54, 12854239),
 (55, 12854450),
 (56, 12856401),
 (57, 12857530),
 (58, 12860191),
 (59, 1

In [0]:
import geopy
def get_distance_matrix(df):
  players = list(df.index)
  distance_matrix = pd.DataFrame(index=players, columns=players)
  coords = np.array(df['coords'])
  for num, pharm in enumerate(players):
    distance_row = [geopy.distance.distance(coords[num], coords[i]).km for i in range(70)]
    distance_matrix.iloc[num] = distance_row
  return distance_matrix

In [0]:
DISTANCE_MATRIX = get_distance_matrix(players_df)

In [0]:
# 최종 Distance Matrix 저장
DISTANCE_MATRIX.to_csv(PATH1+'DISTANCE_MATRIX.csv', encoding='utf-8-sig', index=True)